In [14]:
import re
import spacy
import spacy_transformers
import parse
import pandas as pd
import pdfplumber

In [11]:
#!pip install spacy

In [12]:
from google.colab import files
uploaded = files.upload()

Saving A SALES-S03677-23-24-29-Aug-23.pdf to A SALES-S03677-23-24-29-Aug-23.pdf


In [16]:
d = {}
cnt = 0
file = 'A SALES-S03677-23-24-29-Aug-23.pdf'
#nlp = spacy.load("en_core_web_trf")
nlp = spacy.load("en_core_web_sm")
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()
        for line in text.split('\n'):
          #print(line)
          if line.endswith('Dated'):
            d['Company'] = line.split('Invoice No. Dated')[0]

          if line.split()[0].endswith('/23-24'):
            d['Invoice Number'] = line.split()[0]
            d['Invoice Date'] = line.split()[1]
          if line.startswith('GSTIN/UIN'):
            if cnt == 0:
              d['Providers GSTIN/UIN'] = line.split()[-1]
              cnt += 1
            if cnt == 1:
              d['Recipient GSTIN/UIN'] = line.split()[-1]
          columns = ["Sl", "Description of Goods", "HSN/SAC", "GST", "Net Quantity", "Rate per", "Amount"]
          if line.startswith('1 '):
            split_values = re.split(r'\s(?=\S)', line)
            data = {column: value for column, value in zip(columns, split_values)}
            d.update(data)
          if line.startswith('Total'):
            d['Total Amount'] = line.split()[-1]

    df = pd.DataFrame(d.items())
    df = df.transpose()
    df.reset_index(inplace=False)
    df.to_csv("Report.csv", index=False)
    print(df.head())

                       0               1             2                    3   \
0                 Company  Invoice Number  Invoice Date  Providers GSTIN/UIN   
1  SURAJ DEV TRADING CO.     S03677/23-24     29-Aug-23      29AALFS8665E1ZK   

                    4   5                     6            7         8   \
0  Recipient GSTIN/UIN  Sl  Description of Goods      HSN/SAC       GST   
1      29ACDPN2062N1ZA   1                 TIGER  PLASTIC(RP)  07133100   

             9         10        11            12  
0  Net Quantity  Rate per    Amount  Total Amount  
1             0         %  3,360.00     16,800.00  


In [17]:
#### Following code is not capturing all the required entities

#### USING spacy.load("en_core_web_sm")

In [25]:
d = {}
cnt = 0
file = 'A SALES-S03677-23-24-29-Aug-23.pdf'
#nlp = spacy.load("en_core_web_trf")
nlp = spacy.load("en_core_web_sm")
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()
        for line in text.split('\n'):
          #print(line)
          doc = nlp(str(line))
          for ent in doc.ents:
            #if ent.label_ == 'CARDINAL':
              print(f"Entity in : '{ent.text}' (Label: {ent.label_})")

Entity in : 'SURAJ DEV TRADING CO' (Label: ORG)
Entity in : '20/30,M.G.B' (Label: MONEY)
Entity in : 'APMC YARD' (Label: PERSON)
Entity in : 'Yeshwanthpur' (Label: GPE)
Entity in : '9845741777' (Label: DATE)
Entity in : 'GSTIN/UIN' (Label: ORG)
Entity in : '29AALFS8665E1ZK' (Label: CARDINAL)
Entity in : 'Karnataka' (Label: PERSON)
Entity in : 'Code' (Label: PERSON)
Entity in : '29' (Label: CARDINAL)
Entity in : '23378777,41283488' (Label: CARDINAL)
Entity in : 'Bill' (Label: PERSON)
Entity in : 'KOLLEGAL' (Label: ORG)
Entity in : 'GSTIN/UIN' (Label: ORG)
Entity in : '29ACDPN2062N1ZA' (Label: CARDINAL)
Entity in : 'Karnataka' (Label: PERSON)
Entity in : 'Code' (Label: PERSON)
Entity in : '29' (Label: CARDINAL)
Entity in : 'Sl Description' (Label: PERSON)
Entity in : '1' (Label: CARDINAL)
Entity in : '3,360.00 5' (Label: DATE)
Entity in : 'BAGS 3,360.00' (Label: PERSON)
Entity in : '16,800.00' (Label: CARDINAL)
Entity in : '1.50' (Label: CARDINAL)
Entity in : 'QTL' (Label: GPE)
Entity in

In [26]:
#### USING spacy.load("en_core_web_trf")

In [27]:
d = {}
cnt = 0
file = 'A SALES-S03677-23-24-29-Aug-23.pdf'
nlp = spacy.load("en_core_web_trf")
#nlp = spacy.load("en_core_web_sm")
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()
        for line in text.split('\n'):
          #print(line)
          doc = nlp(str(line))
          for ent in doc.ents:
            #if ent.label_ == 'CARDINAL':
              print(f"Entity in : '{ent.text}' (Label: {ent.label_})")

Entity in : '29-Aug-23' (Label: DATE)
Entity in : 'Yeshwanthpur' (Label: GPE)
Entity in : 'BANGALORE' (Label: GPE)
Entity in : 'Karnataka' (Label: GPE)
Entity in : '29' (Label: CARDINAL)
Entity in : 'Bill' (Label: PERSON)
Entity in : 'KOLLEGAL' (Label: PERSON)
Entity in : 'Karnataka' (Label: GPE)
Entity in : '29' (Label: CARDINAL)
Entity in : '0 %' (Label: PERCENT)
Entity in : '3,360.00' (Label: CARDINAL)
Entity in : '5' (Label: CARDINAL)
Entity in : '3,360.00' (Label: CARDINAL)
Entity in : '16,800.00' (Label: CARDINAL)
Entity in : '1.50 QTL' (Label: MONEY)
Entity in : 'MONG 30KG' (Label: PERSON)
Entity in : '5.00 BAGS' (Label: MONEY)
Entity in : '16,800.00' (Label: CARDINAL)
Entity in : 'INR Sixteen Thousand Eight Hundred' (Label: CARDINAL)
Entity in : 'STATE BANK OF INDIA' (Label: ORG)
Entity in : 'SURAJ DEV TRADING CO' (Label: GPE)
Entity in : 'BANGALORE' (Label: GPE)


In [22]:
#!python -m spacy download en_core_web_trf

2023-09-13 18:58:35.349759: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 989.7 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
